In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [7]:
# Cell 1: Install Dependencies (Updated)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl>=0.8.0" peft accelerate bitsandbytes
!pip install evaluate rouge_score nltk

import torch
import pandas as pd
import os
from datetime import datetime
from datasets import Dataset
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, Any

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mezfggue/unsloth_a52543b0e2ba4e088ab7763fe0adf2a2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-mezfggue/unsloth_a52543b0e2ba4e088ab7763fe0adf2a2
  Resolved https://github.com/unslothai/unsloth.git to commit d1e312dcdc57bf020aa0f6da810226efe79cd69a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6


In [9]:
# Cell 2: Architecture & Classes (Fixed Column Names)
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, Any

# 1. Experiment Log Schema
@dataclass
class ExperimentLog:
    id: str
    model_name: str
    lora_config: Dict[str, Any]
    train_loss: float
    metrics: Dict[str, float]
    timestamp: str

# 2. Strategy Interface
class FineTuningStrategy(ABC):
    @abstractmethod
    def load_model(self, model_name: str):
        pass
    @abstractmethod
    def train(self, dataset, output_dir: str):
        pass

# 3. Dataset Processor (Fixed for 'Questions' column)
class DatasetProcessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def format_prompts(self, examples):
        # FIX: The dataset uses 'Questions' (plural), not 'Question'
        # We add a fallback just in case
        q_col = 'Questions' if 'Questions' in examples else 'Question'
        a_col = 'Answers' if 'Answers' in examples else 'Answer'
        
        questions = examples[q_col]
        answers = examples[a_col]
        
        texts = []
        for q, a in zip(questions, answers):
            text = (
                f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
                f"You are a helpful Bengali AI assistant.<|eot_id|>"
                f"<|start_header_id|>user<|end_header_id|>\n\n{q}<|eot_id|>"
                f"<|start_header_id|>assistant<|end_header_id|>\n\n{a}<|eot_id|>"
            )
            texts.append(text)
        return {"text": texts}

In [10]:
# Cell 3: The Unsloth Engine (Concrete Strategy)
import unsloth
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from typing import Dict, Any
import torch

class UnslothStrategy(FineTuningStrategy):
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.max_seq_length = 2048

    def load_model(self, model_name: str):
        print(f"⚙️ Loading Model: {model_name} via Unsloth...")
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name,
            max_seq_length=self.max_seq_length,
            dtype=None, 
            load_in_4bit=True,
        )
        
        self.model = FastLanguageModel.get_peft_model(
            self.model,
            r=16,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
            lora_alpha=16,
            lora_dropout=0,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=3407,
        )
        print("✅ Model & LoRA Adapters loaded.")

    def train(self, dataset, output_dir: str):
        print("🚀 Starting Training Process...")
        
        trainer = SFTTrainer(
            model=self.model,
            tokenizer=self.tokenizer,
            train_dataset=dataset,
            dataset_text_field="text",
            max_seq_length=self.max_seq_length,
            dataset_num_proc=2,
            packing=False,
            args=TrainingArguments(
                per_device_train_batch_size=2,
                gradient_accumulation_steps=4,
                warmup_steps=5,
                max_steps=60,
                learning_rate=2e-4,
                fp16=not torch.cuda.is_bf16_supported(),
                bf16=torch.cuda.is_bf16_supported(),
                logging_steps=1,
                optim="adamw_8bit",
                weight_decay=0.01,
                lr_scheduler_type="linear",
                seed=3407,
                output_dir=output_dir,
                
                # --- CRITICAL FIXES ---
                remove_unused_columns=True,  # Fix column error
                report_to="none",            # Fix WandB/Login error
            ),
        )
        trainer.train()
        return trainer

print("✅ Unsloth Strategy Updated (Versions & Logs Fixed).")

✅ Unsloth Strategy Updated (Versions & Logs Fixed).


In [11]:
# Cell 4: Tuner Class & Execution
!pip install -q kagglehub

import kagglehub
import pandas as pd
import glob
from datasets import Dataset
from datetime import datetime

# Define the Tuner Class
class LLAMAFineTuner:
    def __init__(self, strategy):
        self.strategy = strategy
    
    def run(self, df, model_name):
        print(f"🚀 Training on {len(df)} rows...")
        
        # Simple Cleanup
        df = df.dropna()
        dataset = Dataset.from_pandas(df)
        
        # Load Model
        self.strategy.load_model(model_name)
        
        # Format Data
        print("⚙️ Formatting prompts...")
        processor = DatasetProcessor(self.strategy.tokenizer)
        
        # --- CRITICAL FIX: Remove original columns to prevent Trainer errors ---
        dataset = dataset.map(
            processor.format_prompts, 
            batched=True, 
            remove_columns=dataset.column_names # <--- Deletes raw columns, keeps only 'text'
        )
        
        # Train
        print("🔥 Starting Training...")
        trainer = self.strategy.train(dataset, "outputs")
        
        # Log
        log = {
            "id": "exp_junior_01",
            "model": model_name,
            "loss": trainer.state.log_history[-1].get('loss', 0),
            "timestamp": str(datetime.now())
        }
        pd.DataFrame([log]).to_csv("LLAMAExperiments.csv", mode='a', index=False)
        return "exp_junior_01"

# --- MAIN EXECUTION ---

# 1. Download Data
print("⬇️ Downloading dataset via KaggleHub...")
dataset_path = kagglehub.dataset_download("raseluddin/bengali-empathetic-conversations-corpus")
print(f"📂 Dataset saved to: {dataset_path}")

# 2. Find CSV
csv_files = glob.glob(f"{dataset_path}/**/*.csv", recursive=True)

if csv_files:
    csv_path = csv_files[0]
    print(f"✅ Found CSV file: {csv_path}")
    df = pd.read_csv(csv_path)
    
    # 3. INITIALIZE GLOBALLY
    strategy = UnslothStrategy() 
    tuner = LLAMAFineTuner(strategy)
    
    # 4. RUN TRAINING
    tuner.run(df, "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")
else:
    print("❌ Critical Error: No CSV file found.")

⬇️ Downloading dataset via KaggleHub...
📂 Dataset saved to: /kaggle/input/bengali-empathetic-conversations-corpus
✅ Found CSV file: /kaggle/input/bengali-empathetic-conversations-corpus/BengaliEmpatheticConversationsCorpus .csv
🚀 Training on 38233 rows...
⚙️ Loading Model: unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit via Unsloth...
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model & LoRA Adapters loaded.
⚙️ Formatting prompts...


Map:   0%|          | 0/37610 [00:00<?, ? examples/s]

🔥 Starting Training...
🚀 Starting Training Process...


Map (num_proc=2):   0%|          | 0/37610 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 37,610 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,1.277100
2,1.080500
3,1.084700
4,0.989000
5,1.188800
6,1.489200
7,1.033900
8,1.141700
9,1.122200
10,1.508100


In [20]:
# Cell 5: Evaluator & Metrics (Safe Mode / Greedy Decoding)
import evaluate
import pandas as pd
from unsloth import FastLanguageModel

class Evaluator:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        print("⏳ Loading metrics...")
        self.bleu = evaluate.load("bleu")
        self.rouge = evaluate.load("rouge")
        
        FastLanguageModel.for_inference(self.model)

    def generate(self, prompt):
        input_text = (
            f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
            f"You are a helpful Bengali AI assistant.<|eot_id|>"
            f"<|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|>"
            f"<|start_header_id|>assistant<|end_header_id|>\n\n"
        )
        
        inputs = self.tokenizer([input_text], return_tensors="pt").to("cuda")
        
        outputs = self.model.generate(
            **inputs, 
            max_new_tokens=64,    # কমিয়ে দিলাম যাতে সে বেশিক্ষণ ভুল না বকে
            use_cache=True,
            pad_token_id=self.tokenizer.eos_token_id,
            
            # --- SAFE MODE (GREEDY) ---
            do_sample=False,      # Creativity বন্ধ (Safety First)
            repetition_penalty=1.1, 
        )
        
        response = self.tokenizer.batch_decode(outputs)[0].split("assistant<|end_header_id|>\n\n")[-1]
        return response.replace("<|eot_id|>", "").strip()

    def evaluate(self, df, num_samples=10):
        print(f"🚀 Evaluating on {num_samples} random samples...")
        q_col = 'Questions' if 'Questions' in df.columns else 'Question'
        a_col = 'Answers' if 'Answers' in df.columns else 'Answer'
        
        test_df = df.sample(n=num_samples)
        preds = []
        for q in test_df[q_col]:
            print(f"   Generating response for: {str(q)[:30]}...") 
            preds.append(self.generate(str(q)))
            
        refs = [[str(a)] for a in test_df[a_col]]
        b_score = self.bleu.compute(predictions=preds, references=refs)
        r_score = self.rouge.compute(predictions=preds, references=[r[0] for r in refs])
        
        print("\n📊 FINAL METRICS:")
        print(f"BLEU: {b_score['bleu']:.4f}")
        print(f"ROUGE-L: {r_score['rougeL']:.4f}")
        
        test_df['Generated_Response'] = preds
        test_df.to_csv("GeneratedResponses.csv", index=False)
        print("✅ GeneratedResponses.csv saved successfully.")

# --- SAFE EXECUTION ---
try:
    my_model = strategy.model
    my_tokenizer = strategy.tokenizer
except NameError:
    print("⚠️ Strategy variable not found, retrieving from Tuner...")
    my_model = tuner.strategy.model
    my_tokenizer = tuner.strategy.tokenizer

evaluator = Evaluator(my_model, my_tokenizer)
evaluator.evaluate(df, num_samples=10)

⏳ Loading metrics...
🚀 Evaluating on 10 random samples...
   Generating response for: আমরা আমাদের মেয়ের স্নাতক ভ্রম...
   Generating response for: ঠিক আমাদের একটি বাচ্চা মেয়ে আ...
   Generating response for: সে একজন মানুষ। যদিও তিনি ইতিমধ...
   Generating response for: আমার প্রাক্তন পিতামাতার সাথে প...
   Generating response for: আমি কর্মক্ষেত্রে পদোন্নতি পেতে...
   Generating response for: আমি খবরে দেখেছি যে স্থানীয় কা...
   Generating response for: হ্যাঁ, কারণ আমি নারীদের ভালোবা...
   Generating response for: আমার প্রাক্তন ৬ মাস আগে আমার স...
   Generating response for: আমি সত্যিই মনে করি এই বছরটি দু...
   Generating response for: এটি সত্যিই অন্যদের প্রয়োজনে স...

📊 FINAL METRICS:
BLEU: 0.0000
ROUGE-L: 0.0000
✅ GeneratedResponses.csv saved successfully.


In [21]:
# Cell 6: Manual Testing
# Let's interact with the model directly!

manual_prompts = [
    "আমার খুব মন খারাপ, আমি কি করতে পারি?",  # (I am very sad, what can I do?)
    "আজকে আমার জন্মদিন, কিন্তু কেউ আমাকে উইশ করেনি।", # (Today is my birthday, but no one wished me.)
]

print("💬 Interactive Test Mode:\n")

for prompt in manual_prompts:
    print(f"👤 User: {prompt}")
    # Generate response using the Evaluator we built
    response = evaluator.generate(prompt)
    print(f"🤖 AI:   {response}")
    print("-" * 50)

💬 Interactive Test Mode:

👤 User: আমার খুব মন খারাপ, আমি কি করতে পারি?
🤖 AI:   এটা স্বাভাবিক! যদি আপনি খুব মন খারাপ হন, তাহলে আপনি নিজে
--------------------------------------------------
👤 User: আজকে আমার জন্মদিন, কিন্তু কেউ আমাকে উইশ করেনি।
🤖 AI:   এটা খোলা হয়েছে! সবাই ভুল ধরে নিচ্ছে যে আপনি অবিলম্�
--------------------------------------------------


In [ ]:
# Cell 7: Final Analysis & Perplexity
import pandas as pd
import math

print("📊 Generating Final Analysis Report...")

# 1. Load the Experiment Log
try:
    log_df = pd.read_csv("LLAMAExperiments.csv")
    latest_run = log_df.iloc[-1]
    
    # 2. Calculate Perplexity
    # Perplexity is defined as e^(loss)
    train_loss = latest_run['loss']
    perplexity = math.exp(train_loss)
    
    print("\n--- Model Performance ---")
    print(f"🆔 Experiment ID: {latest_run['id']}")
    print(f"📉 Final Training Loss: {train_loss:.4f}")
    print(f"🧠 Perplexity Score:    {perplexity:.4f}")
    print("-------------------------")
    
    # 3. Show Deliverables
    print("\n--- Deliverables Check ---")
    print(f"✅ LLAMAExperiments.csv saved ({len(log_df)} records)")
    
    resp_df = pd.read_csv("GeneratedResponses.csv")
    print(f"✅ GeneratedResponses.csv saved ({len(resp_df)} samples)")
    
    print("\nPreview of Generated Responses:")
    print(resp_df[['input_text', 'Generated_Response']].head(2).to_markdown(index=False))

except FileNotFoundError:
    print("❌ Error: Logs not found. Did you run the training cell?")